#### Assigning Libraries

In [1]:
import numpy as np
import pandas as pd
import sklearn 
import joblib
import matplotlib.pyplot as plt
import os

C:\Users\nilas\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


#### Importing Data

In [2]:
project_dir=r"C:\Users\nilas\Documents\Flask_ML_Project"
data_dir="Data"

In [3]:
def get_data(name):
    file_name=f"{name}.csv"
    file_path=os.path.join(project_dir,data_dir,file_name)
    return pd.read_csv(file_path)

In [4]:
train_df=get_data("train")
test_df=get_data("test")
val_df=get_data("val")
flight_df=get_data("flight_price")

#### Exploring Data

In [5]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The dataset contains 10683 rows and 11 columns
- Route and Total_stops have 1 missing value each
- Date and Time fields are having object datatype

#### Handling Duplicates

In [6]:
flight_df.duplicated().sum() #Number of Duplicate Rows

220

In [7]:
#Checking duplicates

flight_df[flight_df.duplicated(keep=False)].sort_values(['Airline','Date_of_Journey','Source','Destination']).head(6)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
9913,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408


#### Cleaning Data

In [11]:
flight_df.select_dtypes(include='O').columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info'],
      dtype='object')

#### Airline

In [12]:
flight_df.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [13]:
(
    flight_df
    .Airline
    .str.replace(" Premium economy","")
    .str.replace(" Business","")
    .str.title()
    .unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

#### Date of Journey

In [14]:
flight_df.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [15]:
pd.to_datetime(flight_df.Date_of_Journey,dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

In [16]:
flight_df.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

#### Departure Time & Arrival Time

In [17]:
#Checking if the field contains anything except number and colon
flight_df.Dep_Time.loc[lambda ser:ser.str.contains("[^0-9:]")]

Series([], Name: Dep_Time, dtype: object)

In [19]:
pd.to_datetime(flight_df.Dep_Time).dt.time

C:\Users\nilas\AppData\Local\Temp\ipykernel_5204\208771400.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flight_df.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

In [20]:
#Checking if the field contains anything except number and colon
flight_df.Arrival_Time.loc[lambda ser:ser.str.contains("[^0-9:]")]

0        01:10 22 Mar
2        04:25 10 Jun
6        10:25 13 Mar
7        05:05 02 Mar
8        10:25 13 Mar
             ...     
10666    19:00 13 Jun
10667    20:20 13 Mar
10672    19:00 28 Jun
10673    04:25 28 May
10674    21:20 13 Mar
Name: Arrival_Time, Length: 4335, dtype: object

#### Duration

In [35]:
(
    flight_df
    .Duration
    .loc[lambda ser: ~ser.str.contains('h')]
)

6474    5m
Name: Duration, dtype: object

In [52]:
(
    flight_df
    .drop(index=[6474])
    .Duration
    .str.split(" ", expand=True)
    .set_axis(['hour','minute'],axis=1)
    .assign(
    hour= lambda df_: (
        df_
        .hour
        .str.replace('h','')
        .astype(int)
        .mul(60)
        ),
    minute=lambda df_: (
        df_
        .minute
        .str.replace('m','')
        .fillna('0')
        .astype(int)
    )
    )
    .sum(axis=1)
    .rename('duration_minutes')
    .to_frame()
    .join(flight_df.Duration)
)

,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


#### Total Stops

In [62]:
(
    flight_df
    .Total_Stops
    .replace("non-stop","0")
    .str.replace(" stops?","", regex=True)
    .pipe(lambda ser: pd.to_numeric(ser))  #nan values can't be converted to int by astype
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

#### Creating Cleaning function

In [54]:
def convert_minute(ser):
    return(
        ser
        .str.split(" ", expand=True)
        .set_axis(['hour','minute'],axis=1)
        .assign(
        hour= lambda df_: (
            df_
            .hour
            .str.replace('h','')
            .astype(int)
            .mul(60)
            ),
        minute=lambda df_: (
            df_
            .minute
            .str.replace('m','')
            .fillna('0')
            .astype(int)
        )
        )
        .sum(axis=1)
    )
    

In [65]:
def clean_data(df):
    return (df
            .drop(index=[6474])
            .assign(**{
                col:df[col].str.strip()
                for col in df.select_dtypes(include="O").columns #Dictionary Comprehension & Unpacking (using **)
            })
            .rename(columns=str.lower)
            .assign(
                airline=lambda df_: (df_
                                 .airline
                                 .str.replace(" Premium economy","")
                                 .str.replace(" Business","")
                                 .str.title()),
                date_of_journey=lambda df_:(pd.to_datetime(df_.date_of_journey,dayfirst=True)),
                dep_time=lambda df_:(pd.to_datetime(df_.dep_time).dt.time),
                arrival_time=lambda df_:(pd.to_datetime(df_.arrival_time).dt.time),
                duration=lambda df_: df_.duration.pipe(convert_minute),
                total_stops=lambda df_: (
                df_
                .total_stops
                .replace("non-stop","0")
                .str.replace(" stops?","", regex=True)
                .pipe(lambda ser: pd.to_numeric(ser))),
                additional_info=lambda df_: (
                df_
                .additional_info
                .replace('No info','No Info'))
                            )
            .drop(columns='route')
           )

In [69]:
flight_df_cln=clean_data(flight_df)
flight_df_cln.head()

C:\Users\nilas\AppData\Local\Temp\ipykernel_5204\2803177302.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time=lambda df_:(pd.to_datetime(df_.dep_time).dt.time),
C:\Users\nilas\AppData\Local\Temp\ipykernel_5204\2803177302.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time=lambda df_:(pd.to_datetime(df_.arrival_time).dt.time),


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
